# 4. Blackjack
Blackjack, also known as 21, is a card game where players try to get as close to 21 points as possible without going over.<br>
This program uses images drawn with text characters, called *ASCII art*.<br>
American Standard Code for Information Interchange (ASCII) is a mapping of text characters to numeric codes that computers used before Unicode replaced it.<br>
The playing cards in this program are an example of ASCII art:
```
 ___   ___
|A  | |10 |
| ♣ | | ♦ |
|__A| |_10|
```

## The Program in Action
When you run *blackjack.py*, the output will look like this:
```
Rules:
    Try to get as close to 21 without going over.
    Kings, Queens, and Jacks are worth 10 points.
    Aces are worth 1 or 11 points.
    Cards 2 through 10 are worth their face value.
    (H)it to take another card.
    (S)tand to stop taking cards.
    On your first play, you can (D)ouble down to increase your bet
    but must hit exactly one more time before standing.
    In case of a tie, the bet is returned to the player.
    The dealer stops hitting at 17.

Money: 5000
How much do you bet? (1-5000, or QUIT)
> 400
Bet: 400

DEALER: ???
 ___   ___
|## | |2  |
|###| | ♥ |
|_##| |__2|

PLAYER: 17
 ___   ___
|K  | |7  |
| ♠ | | ♦ |
|__K| |__7|

(H)it, (S)tand, (D)ouble down
> h
You drew a 4 of ♦.
--snip--
DEALER: 18
 ___   ___   ___
|K  | |2  | |6  |
| ♦ | | ♥ | | ♠ |
|__K| |__2| |__6|

PLAYER: 21
 ___   ___   ___
|K  | |7  | |4  |
| ♠ | | ♦ | | ♦ |
|__K| |__7| |__4|
You won $400!
--snip--
```

## How It Works
The card suit symbols do not exist on your keyboard, which is why we call the *chr()* function to create them.<br>
The integer passed to *chr()* is called a Unicode *code point*, a unique number that identifies a character according to the Unicord standard.<br>
Unicode is often misunderstood.<br>
However Ned Batchelder's 2012 PyCon US talk "Pragmatic Unicode, or How Do I Stop the Pain?" is an excellent introduction to Unicode, and you can find it at https://youtu.be/sgHbC6udIqc/.<br>


In [ ]:
"""Blackjack by Al Sweigart al@inventwithpython.com
The clasic card game known as 21. (This version does not have
splitting or insurance.)
More info at: https://en.wikipedia.org/wiki/Blackjack
View this code at https://nostarch.com/big-book-small-python-projects
Tags: large, game, card game"""

import random, sys

# Set up the constants:
HEARTS   = chr(9829) # Character 8929 is the symbol for hearts.
DIAMONDS = chr(9830) # Character 8930 is the symbol for diamonds.
SPADES   = chr(9824) # Character 9824 is the symbol for spades.
CLUBS    = chr(9827) # Character 9827 is the symbol for clubs.
# A list of chr() codes can be found at https://inventwithpython.com/charactermap
BACKSIDE = 'backside'


def main():
    print('''Blackjack by Al Sweigart - al@inventwithpython..com)

    Rules:
        Try to get as close to 21 without going over.
        Kings, Queens, and Jacks are worth 10 points.
        Aces are worth 1 or 11 points.
        Cards 2 through 10 are worth their face value.
        (H)it to take another card.
        (S)tand to stop taking cards.
        On your first place, you can (D)ouble down to increase your bet
        but must hit exactly one more time before standing.
        In case of a tie, the bet is returned to the player.
        The dealer stops hitting at 17 or greater.''')

    money = 5000
    while True: # Main game loop
        # Check if the player has run out of money:
        if money <= 0:
            print("You are broke!")
            print("Good thing you weren't playing with real money.")
            print("Thanks for playing!")
            sys.exit()
        
        # Let the player enter their bet for this round:
        print(f'Money: {money}')
        bet = getBet(money)

        # Give the dealer and player two cards from the deck each:
        deck = getDeck()
        dealerHand = [deck.pop(), deck.pop()]
        playerHand = [deck.pop(), deck.pop()]
        
        # Handle player actions:
        print(f'Bet: {bet}')
        while True: # Keep looping until player stands or busts.
            displayHands(playerHand, dealerHand, False)
            print()

            # Check if the player has bust:
            if getHandValue(playerHand) > 21:
                break

            # Get the player's move, either (H)it, (S)tand, (D)ouble:
            move = getMove(playerHand, money - bet)

            # Handle the player's actions:
            if move == 'D':
                # Player is doubling down, they can increase their bet:
                additionalBet = getBet(min(bet, (money - bet)))
                bet += additionalBet
                print(f'Bet increased to {bet}')
                print(f'Bet: {bet}')

                if move in ('H', 'D'):
                    # Hit/doubling down, they can increase their bet:
                    newCard = deck.pop()
                    rank, suit = newCard
                    print(f'You drew a {rank} of {suit}.')
                    playerHand.append(newCard)

                    if getHandValue(playerHand) > 21:
                        # The player has busted:
                        continue

                if move in ('S', 'D'):
                    # Stand/doubling down stops the player's turn.
                    break
            
        # Handle the dealer's actions:
        if getHandValue(playerHand) <=21:
            while getHandValue(dealerHand) < 17:
                # The dealer hits: 
                print('Dealer hits...')
                dealerHand.append(deck.pop())
                displayHands(playerHand, dealerHand, False)

                if getHandValue(dealerHand) > 21:
                    break   # The dealer has busted.
                input('Press Enter to continue...')
                print('\n\n')

        # Show the final hands:
        displayHands(playerHand, dealerHand, True)

        playerValue = getHandValue(playerHand)
        dealerValue = getHandValue(dealerHand)
        # Handle whether the player won/lost/tied:
        if dealerValue > 21:
            print(f'Dealer busts! You win ${bet}')
            money += bet
        elif (playerValue > 21) or (playerValue < dealerValue):
            print('You lost!')
            money -= bet
        elif playerValue > dealerValue:
            print(f'You won ${bet}')
            money += bet
        elif playerValue == dealerValue:
            print("It's a tie, the bet has been refunded.")

        input('Press Enter to continue...')
        print('\n\n')


def getBet(maxBet):
    """Ask the player how much they want to bet for this round."""
    while True: # Keep asking until they enter a valid amount.
        print(f'How much are you betting? (1 - {maxBet}, or QUIT)')
        bet = input('> ').upper().strip()
        if bet == 'QUIT':
            print('Thanks for playing!')
            sys.exit
        
        if not bet.isdecimal():
            continue # If the player did not enter a number, ask again.

        bet = int(bet)
        if 1 <= bet <= maxBet:
            return bet # Player entered a valid bet.


def getDeck():
    """Return a list of (rank, suit) tuples for all 52 cards."""
    deck = []
    for suit in (HEARTS, DIAMONDS, SPADES, CLUBS):
        for rank in range(2,11):
            deck.append((str(rank), suit)) # Add the numbered cards
        for rank in ('J', 'Q', 'K', 'A'):
            deck.append((rank, suit)) # Add the face and ace cards.
    random.shuffle(deck)
    return deck


def displayHands(playerHand, dealerHand, showDealerHand):
    """Show the player's and dealer's cards. 
    Hide the dealer's first card if showDealerHand is False."""
    print()
    if showDealerHand:
        print(f'DEALER: {getHandValue(dealerHand)}')
        displayCards(dealerHand)
    else:
        print('DEALER: ???')
        # Hide the dealer's first card:
        displayCards([BACKSIDE] + dealerHand[1:])

    # Show the player's cards:
    print(f'PLAYER: {getHandValue(playerHand)}')
    displayCards(playerHand)


def getHandValue(cards):
    """Returns the value of cards. Face cards are worth 10, Aces are 
    worth 11 or 1 (This function picks the most suitable ace value)."""
    value = 0
    numberOfAces = 0

    # Add the value for the non-ace cards:
    for card in cards:
        rank = card[0] # card is a ruple like (rank, suit)
        if rank == 'A':
            numberOfAces += 1
        elif rank in ('K', 'Q', 'J'): # Face cards are woth 10 points.
            value += 10
        else:
            value += int(rank) # Numbered cards are woth their number.

    # Add the value for the aces:
    value += numberOfAces   # Add 1 per ace.
    for i in range(numberOfAces):
        # If another 10 can be added without busting, do so:
        if value + 10 <= 21:
            value += 10

    return value


def displayCards(cards):
    """Display all the cards in the cards list."""
    rows = ['', '', '', '', ''] # The text to display on each row.

    for i, card in enumerate(cards):
        rows[0] += ' ___  ' # Print the top line of the cards.
        if card == BACKSIDE:
            # Print a card's back:
            rows[1] += '|## | '
            rows[2] += '|###| '
            rows[3] += '|_##| '
        else:
            # Print the card's front:
            rank, suit = card # The card is a tuple data structure.
            rows[1] += f'|{rank.ljust(2)} | '
            rows[2] += f'| {suit} | '
            rows[3] += f'|_{rank.rjust(2, "_")}| '

    # Print each row on the screen:
    for row in rows:
        print(row)
def getMove(playerHand, money):
    """Asks the player for their move, and returns 'H' for hit, 'S' for
    stand, and 'D' for double down."""
    while True: # Keep looping until the player enters a correct move.
        # Determine what moves the player can mak:
        moves = ['(H)it', '(S)tand']

        # The player can double down on their first move, which we can 
        # tell because they'll have exactly two cards:
        if len(playerHand) == 2 and money > 0:
            moves.append('(D)ouble down')

        # Get the player's move:
        movePrompt = ', '.join(moves) + '> '
        move = input(movePrompt).upper()
        if move in ('H', 'S'):
            return move # Player has entered a valid move.
        if move == 'D' and '(D)ouble down' in moves:
            return move # Player has entered a valid move.
        
        
# If the program is run (instead of imported), run the game:
if __name__ == '__main__':
    main()

After entering the source code and runninng it a few times, try making experimental changes to it.<br>
Blackjack has several custom rules that you could implement.<br>
For example, if the first two cards have the same value, a player can split them into two hands and wager on them separately.<br>
Also, if the player receives a "blackjack" (Ace Spades & Black Jack) for their first two cards, the player wins a ten-to-one payout.<br>
You can find out more about the game from *https://en.wikipedia.org/wiki/Blackjack*.



### Implementing Split
The conditions for the player to split their hand are:
* Both cards dealt are the same rank, and
* Player has enough money to wager the same bet.

To get the individual cards we need to split playerHand into its two cards.
```
playerHandFirstCard = playerHand[0]
playerHandSecondCard = playerHand[1]
```

We can check if the cards are the same rank.
```
if playerHandFirstCard[rank] == playerHandSecondCard[rank] AND if money > 2 * bet:
    player is able to make the move to split
```

After this check has been done we proceed with using the first card split from the initial hand and draw a card from the deck.
```
splitOne = [playerHandFirstCard, deck.pop()]
splitTwo = [playerHandSecondCard, deck.pop()]
```

Proceed until the hand has been completed then we move on to the second split.<br>

For this we can show both hands, both values, and the total money won/returned/lost after the dealer's hand has been shown. 


### Implementing Blackjack payout
The conditions for a Blackjack payout are:
* Ace of Spades + Jack of Spades, OR
* Ace of Spades + Jack of Clubs

Using sorted(*list*) we do not need to check if Ace Spades is first or second in the player's hand.
```
blackjackClubs = [(A,SPADES), (J, CLUBS)]
blackjackSpades = [(A,SPADES), (J, SPADES)]

if sorted(playerHand) == sorted(blackjackClubs) OR sorted(playerHand) == sorted(blackjackSpades)
```

Once resolved we can perform the payout.
```
money += bet * 1.1
```

## Exploring the Program
Try to find answers to the following questions.<br>
Experiment with some modifications to the code and rerun the program to see what effect the changes have.


### 1. How can you make the player start with a different amount of money?
We can change the `money` variable at the start of the program to adjust the amount we start with.

### 2. Hows does the program prevent the player from betting more money than they have?
In the object `getBet`, there is the check:
```
if 1 <= bet <= maxBet
```

This ensures that the value cannot be greater than the money we have.

### 3. Hows does the program represent a single card?
A single card is represented as a `string` and ASCII symbol.

### 4. How does the program represent a hand of cards?
A hand of cards is represented as a list of cards.

### 5. What do each of the strings in the `rows` list (created on line 197) represent?
The list of rows represents the new line of the ASCII card.

### 6. What happens if you delete/comment out `random.shuffle(deck)` on line 148?
By removing `random.shuffle(deck)`, the generated cards will be the same everytime, i.e. 2 Hearts, 3 Hearts, 4 Hearts, etc.<br>
We will get the same *unshuffled* deck when running the program.

### 7. What happends if you change `money -= bet` on line 112 to `money += bet`?
By changing the symble from `-=` to `+=` we effectively always add the bet to our money amount if we bust or when the dealer has a better hand.

### 8. What happens when `showDealerHand` in the `displayHands()` function is set to `True`? What happens when it is `False`?
If the `showDealerHand` is set to `True`, we are able to see what the dealer's second card is.<br>
By setting it to default to `False`, we do not see the second card until the player has finished their hand.